In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
import pandas as pd
from xbbg import blp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from datetime import timedelta as td
import statsmodels.api as sm
import re
import matplotlib.image as image
from ipywidgets import interact, IntSlider, Checkbox, Dropdown, Output, HBox, VBox, interactive, interactive_output, ToggleButton,Text, Button, DatePicker, IntText, ToggleButtons, RadioButtons,SelectMultiple
from IPython.display import display, clear_output
import itertools
from scipy import stats
from scipy.optimize import minimize 
from scipy.special import ndtr

spot_library = {'eur_6':['EUSA', ' Curncy'],'eur': ['EESWE', ' Curncy'], 'usd': ['USOSFR', ' Curncy'], 
                'gbp': ['BPSWS', ' Curncy'],'chf': ['SFSNT', ' Curncy'],'sek': ['SKSW', ' Curncy'],
                 'nok': ['NKSW', ' Curncy'], 'hkd': ['HDSW', ' Curncy'],'czk': ['CKSW', ' Curncy'],
                 'pln': ['PZSW', ' Curncy'],'ils':['ISSW', ' Curncy'],  'cad':['CDSW', ' Curncy'], 
                 'jpy':['JYSO', ' Curncy'], 'aud': ['ADSW', ' Curncy'],'sgd':['SDSW', ' Curncy'],
                'krw': ['KWSWNI', ' Curncy'],
                'zar': ['SASW', ' Curncy'],
                'nzd': ['NDSW', ' Curncy'],
                'mxn': ['MPSW', ' Curncy']} 

forward_library = {'eur_6': ['EUSA', ' Curncy'], 
                 'eur': ['S0514FS ', ' BLC Curncy'], 
                 'usd': ['S0490FS ', ' BLC Curncy'], 
                 'gbp': ['S0141FS ', ' BLC Curncy'],
                 'chf': ['S0234FS ', ' BLC Curncy'],
                 'sek': ['SD0020FS ', ' BLC Curncy'],
                 'nok': ['SD0313FS ', ' BLC Curncy'],
                 'hkd': ['HDFS', ' Curncy'],
                 'czk': ['S0320FS ', ' BLC Curncy'],
                 'pln': ['S0323FS ', ' BLC Curncy'],
                 'ils': ['ISFS', ' Curncy'],
                 'cad': ['CDFS', ' Curncy'],
                 'jpy': ['S0195FS ', ' BLC Curncy'],
                 'aud': ['SD0302FS ', ' BLC Curncy'],
                'sgd': ['SDFS', ' Curncy'],
                'krw': ['S0205FS ', ' BLC Curncy'],
                'zar': ['SAFS', ' Curncy'],#
                'nzd': ['SD0015FS ', ' BLC Curncy'],
                'mxn': ['SD0083FS ', ' BLC Curncy']} 

################################################# SWAP STRUCTURE BUILDERS ################################################

def cut(some):
    x = 0
    y = some[0]
    while y.isdigit():
        y=some[x]
        x+=1
    return some[:x],some[x:]


def f(tenor):
    x = re.findall(r'\d+',tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 >0:
            num+='1'
        num+=chr(64+(int(x)%12))
    else:
        if len(x) == 1:
            num+='0'
        num +=x
    return num

def t(tenor):
    x = re.findall(r'\d+',tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 >0:
            num+='1'
        else:
            num+='0'
        num+=chr(64+(int(x)%12))
    else:
        if len(x) == 1:
            num+='0'
        num +=x
    return num


def spot_ticker(dex,tenor):
    if dex.lower() == 'mxn':
        y = int(tenor[:-1]) * 13
        num = f'{y//12}{chr(64+(y%12))}'
    else:
        num = tenor[:-1]
    return f'{spot_library[dex][0]}{num}{spot_library[dex][1]}'

def forward_ticker(dex,fwd):
    dex = dex.lower()
    fwd = fwd.lower()
    old = ['eur_6','hkd','ils','cad','sgd','zar']
    
    if cut(fwd)[0] == '0y':
        return spot_ticker(dex,cut(fwd)[1])
    elif dex == 'eur_6':
        F,T = f(cut(fwd)[0]),t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    elif dex in old:
        F,T = t(cut(fwd)[0]),t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    else:
        
        return f'{forward_library[dex][0]}{fwd.upper()}{forward_library[dex][1]}'


def swap_structure(dex,structure,start, end = 'today',bps = True):
    f_fly = structure.count('/') == 2
    f_crv = structure.count('/') == 1  
    fly = structure.count('s') == 3
    crv = structure.count('s') == 2
    out = max(2 -sum([i.isalpha() for i in structure]),0)
    
    if f_fly or f_crv:
        legs = [forward_ticker(dex,i) for i in structure.split('/')]
    elif fly or crv:
        legs = [spot_ticker(dex,i) for i in [i+'Y' for i in structure.split('s') if i.isdigit()]]
    else:
        legs = forward_ticker(dex,('0Y'*out) + structure)
        
    df = blp.bdh(legs, flds='px_last', start_date=start,end_date=end).fillna(method ='bfill')  *(100 if bps else 1)
    s  = pd.DataFrame({})
    if f_fly or fly:
        x = (2 * df.iloc[:,1]) - (df.iloc[:,0] + df.iloc[:,2])
    elif f_crv or crv:
        x = df.iloc[:,1] - df.iloc[:,0]
    else:
        x = df.iloc[:,0]
        
    s[f'{dex.upper()} {structure}'] = x
    return s

################################################# BOND STRUCTURE BUILDERS ################################################
def bond_ticker(bond_name, bond_tenor):
    cmb = {'ust':1,'dbr':2,'spgb':8,'frtr':4,'btp':5,'ukt':6}
    return f'RV000{cmb[bond_name]}P {bond_tenor.upper()} BLC Curncy'

def bond_structure(bonds,structure, start, end = 'today'):
    if '-' in bonds:
        s = pd.DataFrame({})
        bond1,bond2 = bonds.split('-')
        s[f'{bonds.upper()} {structure}'] = bond_structure(bond1,structure, start, end)[f'{bond1.upper()} {structure}'] - bond_structure(bond2,structure, start, end)[f'{bond2.upper()} {structure}']
        return s
    else:
        if 'y' in structure.lower(): #bond o/r
            tickers = bond_ticker(bonds.lower(), structure)
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = [f'{bonds.upper()} {structure}']
            return base
        
        elif structure.lower().count('s') == 2: #bond curve
            tickers = [bond_ticker(bonds.lower(), f'{i}y') for i in structure.lower().split('s') if i != '']
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = tickers
            s = pd.DataFrame({})
            s[f'{bonds.upper()} {structure}'] = base[tickers[1]] - base[tickers[0]]
            return s
        
        elif structure.lower().count('s') == 3: # bond fly
            tickers = [bond_ticker(bonds.lower(), f'{i}y') for i in structure.lower().split('s') if i != '']
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = tickers
            s = pd.DataFrame({})
            s[f'{bonds.upper()} {structure}'] = (2*base[tickers[1]]) - (base[tickers[0]] + base[tickers[2]])
            return s

################################################# FUTURES STRUCTURE BUILDERS ################################################

def fut_ticker(dex, contract, name = False):
    x  = ''
    if dex.lower()[:3] == 'eur':
        x = 'ER'
    elif dex.lower() == 'usd':
        x = 'SFR'
    elif dex.lower() == 'gbp':
        x = 'SFI'
    
    if name:
        return x
    else:
        return f'{x}{contract} Comdty'   
           
    
def fut_structure(dex,structure, start, end = 'today'):
    name = fut_ticker(dex.lower(), 0,True)
    
    if structure.lower().count('s') == 2: #fut curve
        tickers = [fut_ticker(dex.lower(), int(i)) for i in structure.lower().split('s') if i != '']
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{name}{structure.replace("s","")}'] = base[tickers[0]] - base[tickers[1]]
        return s
        
    elif structure.lower().count('s') == 3: # fut fly
        tickers = [fut_ticker(dex.lower(), int(i)) for i in structure.lower().split('s') if i != '']
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{name}{structure.replace("s","")}'] = (2*base[tickers[1]]) - (base[tickers[0]] + base[tickers[2]])
        return s
    else:
        tickers = fut_ticker(dex.lower(), structure)
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = [f'{name}{structure}']
        return base

################################################# ASW STRUCTURE BUILDERS ################################################

def get_asw(bond_name,tenor,start, end ='today',euro = 'estr'):
    asso_swap = {'ust':'usd','dbr':'eur','spgb':'eur','frtr':'eur','btp':'eur','ukt':'gbp'}
    
    if asso_swap[bond_name] == 'eur' and euro != 'estr':
        tickers = [bond_ticker(bond_name,tenor), spot_ticker(asso_swap[bond_name]+'_6',tenor)]
    else:
        tickers = [bond_ticker(bond_name,tenor), spot_ticker(asso_swap[bond_name],tenor)]
    base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
    base.columns = tickers
    s = pd.DataFrame({})
    s[f'{tenor.upper()} {bond_name.upper()} ASW'] = base[tickers[1]] - base[tickers[0]]
    return s

def asw_structure(bonds,structure, start, end = 'today',euro = 'estr'):
    if '-' in bonds:
        s = pd.DataFrame({})
        bond1,bond2 = bonds.split('-')
        s[f'{bonds.upper()} ASW {structure}'] = asw_structure(bond1,structure, start, end,euro)[f'{bond1.upper()} ASW {structure}'] - asw_structure(bond2,structure, start, end,euro)[f'{bond2.upper()} ASW {structure}']
        return s
    else:
        if 'y' in structure.lower(): #asw o/r
            return get_asw(bonds,structure,start, end,euro)
        
        elif structure.lower().count('s') == 2: #asw curve
            legs = [ f'{i}y' for i in structure.lower().split('s') if i != '']
            short_leg, long_leg = [get_asw(bonds,i,start,end,euro) for i in legs]
            df = short_leg.join(long_leg, how = 'inner')
            s = pd.DataFrame({})
            s[f'{bonds.upper()} ASW {structure}'] = df.iloc[:,1]  - df.iloc[:,0]  
            return s
        
        elif structure.lower().count('s') == 3: # asw fly
            legs = [ f'{i}y' for i in structure.lower().split('s') if i != '']
            short_leg, belly ,long_leg = [get_asw(bonds,i,start,end,euro) for i in legs]
            short_leg = short_leg.join(belly, how = 'inner')
            df = short_leg.join(long_leg, how = 'inner')
            s = pd.DataFrame({})
            s[f'{bonds.upper()} ASW {structure}'] = (2 * df.iloc[:,1])  - (df.iloc[:,0] + df.iloc[:,2])  
            return s
        

######################################### TICKER HANDLERS #################################################################

def get_other(ticker,attribute,title,start,end='today' ):
    if 'price' in attribute.lower():
        attribute = 'px_last'
    else:
        attribute = 'YLD_YTM_MID'
    base = blp.bdh(ticker,attribute,start,end)
    base.columns = [title]
    return base

######################################### INVOICE SPREAD STRUCTURE BUILDERS ################################################


def get_ivsp(country,start, end ='today',which = 'TUA',euro ='estr'):
    tkr = {'USD':['TUAISPS','3YAISPS','FVAISPS','TYAISPS','UXYAISPS','USAISPS','WNAISPS'],
           'GER':['DUAISP','OEAISP','RXAISP','UBAISP'],
           'ITA':['BTAISP','IKAISP'],
           'FRA':['OATAISP'],
           'GBP':['GAISPO']}
    
    if country.upper() == 'USD' or country.upper() == 'GER' or country.upper() == 'ITA' :
        for i in tkr[country.upper()]:
            if which in i:
                future = i
    else:
        future = tkr[country.upper()][0]
    
    if country.lower() in ['ger','ita','fra'] and euro == 'estr':
        future += 'E'
    final_ticker = f'{future} Comdty'
    base = blp.bdh(final_ticker,'px_last',start,end)
    base.columns = [f'{future} Comdty']    
    return base
                
######################################## MEETING DATES HANDLER ##################################################################
    
def md_swap_ticker(bank, meeting,m_of_m =False):
    central_bank = {'FOMC':['USSOFED',' Curncy'],
                'ECB':['EESF','A Curncy'],
                'MPC':['GPSF','A Curncy']}
    ticker = f'{central_bank[bank.upper()][0]}{meeting}{central_bank[bank.upper()][1]}' 
    if m_of_m:
        return blp.bdp(ticker, 'SW_EFF_DT').values.flatten()[0].strftime('%b')
    else:
        return ticker
    
def get_md_swap(bank,structure,start,end  = 'today'):
    if structure.lower().count('s') == 2:
        tickers = [md_swap_ticker(bank.upper(),int(i)) for i in structure.lower().split('s') if i != ''] 
        lbls = ''.join([md_swap_ticker(bank.upper(),int(i), True) for i in structure.lower().split('s') if i != ''])
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{lbls} {bank.upper()}'] = base[tickers[1]] - base[tickers[0]]
        return s
        
    if structure.lower().count('s') == 3:
        tickers = [md_swap_ticker(bank.upper(),int(i)) for i in structure.lower().split('s') if i != '']
        lbls = ''.join([md_swap_ticker(bank.upper(),int(i), True) for i in structure.lower().split('s') if i != ''])            
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{lbls} {bank.upper()}'] = (2*base[tickers[1]] )- (base[tickers[0]] + base[tickers[2]])
        return s
    else:
        tickers = md_swap_ticker(bank.upper(), structure)
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = [f'{md_swap_ticker(bank.upper(), structure,True)}{bank.upper()}']
        return base

    
################################################# FX HANDLER ############################################################
def get_fx_pair(pair,start,end='today'):
    ticker = f'{pair.upper()} Curncy'
    base = blp.bdh(ticker,'px_last',start,end)
    base.columns = [pair.upper()]
    return base

######################################## BOND FUTURE HANDLER ##################################################################
def get_bondf_ticker(name):
    tic_dic = {'DU':'DUA Comdty','OE':'OEA Comdty','RX': 'RXA Comdty','UB': 'UBA Comdty',
                'BTS': 'BTSA Comdty','IK': 'IKA Comdty',
                'OAT': 'OATA Comdty',
                'G':'G A Comdty',
                'TU': 'TUA Comdty','FV': 'FVA Comdty', 'TY': 'TYA Comdty','WN': 'WNA Comdty','UXY': 'UXYA Comdty'}
    return tic_dic[name.upper()]
    

def get_bond_fut(_str, start,end = 'today', fld = 'yield'):
    fld = 'YLD_YTM_MID' if fld.lower() == 'yield' else 'PX_LAST' 
    _str = _str.upper()
    label = _str.split('/')
    legs = [get_bondf_ticker(i) for i in label]
    bps = 100 if fld == 'YLD_YTM_MID' else 1
    base  = blp.bdh(legs,fld,start,end) * bps
    base.columns = label
    s = pd.DataFrame({})
    
    if _str.count('/') == 2:
        s[_str] = 2 * base[label[1]] - (base[label[0]] + base[label[2]]) 
            
    elif _str.count('/') == 1:
        s[_str] =  base[label[1]] - base[label[0]] 
                                        
    else:
        s = base
        
    return s
    

######################################## INFL SWAP HANDLER ##################################################################



######################################## MASTER HANDLER ##################################################################


def function_for_bb(trades,_from,_to):
    master_c = pd.DataFrame({})
    for i in trades.keys():
        if len(trades[i]) != 0:
            for j in trades[i]:
                if i == 'SWAP':
                    h = swap_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)                                
                elif i == 'FUTURE':
                    h = fut_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)                              
                elif i == 'BOND':
                    h = bond_structure(bonds = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                elif i ==  'ASW':
                    h = asw_structure(bonds = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to, euro = j[2].lower())
                elif i == 'XMKT SWAP':
                    h1 = swap_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                    h2 = swap_structure(dex = j[2].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                    h_int = h1.join(h2, how = 'inner')
                    h = pd.DataFrame(h_int.iloc[:,0] - h_int.iloc[:,1], columns = [f'{j[0]}-{j[2]} {j[1]}'])
                elif i == 'MD SWAP':
                    h = get_md_swap(bank = j[0].upper(),structure=j[1].lower() ,start = _from , end = _to)
                elif i ==  'BOND FUTURE':
                    h = get_bond_fut(_str = j[0],start = _from , end = _to,fld =  j[1])
                elif i == 'FX':
                    h = get_fx_pair(pair = j[0].upper(),start = _from , end = _to)
                elif i == 'OTHER/CIX':
                    h = get_other(ticker = j[0], attribute= j[1], title = j[2], start = _from , end = _to)            

                if master_c.shape == (0,0):
                    master_c = h
                else:
                    master_c = master_c.join(h, how = 'inner')
    return master_c
                            
                            
    

In [3]:

clrs = ['whites','reds','greens','blues']
def fut_presets(dex,clr):
    color = {key : range(1+(clrs.index(key)*4),5+(clrs.index(key)*4)) for key in clrs}
    fin_clr = []
    for i in clr.lower().split(', '):
        fin_clr += list(color[i])
    for i in fin_clr:
        yield (dex.upper(), str(i))

clr_dd = [', '.join([m.capitalize() for m in j]) for i in range(1,len(clrs)+1) for j in itertools.combinations(clrs, i) ]


sp_or_fw = ['Spot Tenors', 'Forward Gaps']
yc = ['(up to 10y)','(up to 15y)','(up to 30y)','(up to 50y)']
srt = ['Spot Curves', 'Spot Flys', 'Forward Curves']
swap_ops = [f'{i} {j}' for i in sp_or_fw for j in yc] + srt
bond_ops = ['Outright', 'Curves', 'Flys']

def swap_presets(dex, build):
    
    spot_front = [f'{i}y' for i in range (1,11)]
    spot_mid = spot_front + ['12y','15y']
    spot_back = spot_mid + ['20y','25y','30y']
    spot_ultraback =spot_back + ['40y','50y']
    
    fwds_front = [f'{i}y1y' for i in range (1,10)]
    fwds_mid = fwds_front +['10y2y','12y3y']
    fwds_back = fwds_mid + ['15y5y','20y5y','25y5y']
    fwds_ultraback = fwds_back + ['30y10y','40y10y']
    
    spot_curves = ['1s2s','2s5s','2s10s','5s10s','5s30s','10s30s']
    spot_flys = ['1s2s3s','1s3s5s', '2s5s10s', '5s10s30s']
    fwds_curves = ['1y1y/5y5y','1y2y/1y10y']
    answer = []
    if 'spot' in build.lower():
        if '10y' in build.lower():
            answer = spot_front
        elif '15y' in build.lower():
            answer = spot_mid
        elif '30y' in build.lower():
            answer = spot_back
        elif '50y' in build.lower():
            answer = spot_ultraback
        elif 'curves' in build.lower():
            answer = spot_curves
        elif 'flys' in build.lower():
            answer = spot_flys 
        else:
            answer = spot_curves + spot_flys
    else:
        if '10y' in build.lower():
            answer = fwds_front
        elif '15y' in build.lower():
            answer = fwds_mid
        elif '30y' in build.lower():
            answer = fwds_back
        elif '50y' in build.lower():
            answer = fwds_ultraback
        else:
            answer = fwds_curves
    for i in answer:
        yield(dex.upper(),i)
        
def bond_spline_presets(bond,build):
    answer = {'Outright': ['2y','5y','10y','30y'],
             'Curves': ['2s5s','2s10s','5s10s','5s30s','10s30s'], 
              'Flys':['2s5s10s', '5s10s30s']}
    for i in answer[build]:
        yield (bond.upper(), i)
        
def asw_spline_presets(bond,build,euro = 'ESTR'):
    answer = {'Outright': ['2y','5y','10y','30y'],
             'Curves': ['2s5s','2s10s','5s10s','5s30s','10s30s'], 
              'Flys':['2s5s10s', '5s10s30s']}
    for i in answer[build]:
        yield (bond.upper(), i,euro)

def custom_presets(broker):
    if broker == "MR Presets":
        return {'SWAP': [('USD','2y1y/3y2y/5y2y'),('GBP','2y1y/3y2y/5y2y'),('EUR','2y1y/3y2y/5y2y'),('EUR_6','2y1y/3y2y/5y2y'),
                         ('USD','5y2y/7y3y'),('GBP','5y2y/7y3y'),('EUR','5y2y/7y3y'),('EUR_6','5y2y/7y3y'),
                         ('USD','5s7s10s'),('GBP','5s7s10s'),('EUR','5s7s10s'),('EUR_6','5s7s10s'),
                        ]}
        

In [4]:
trades = {'SWAP':[],'FUTURE':[],'BOND':[], 'BOND FUTURE': [],'ASW':[],'MD SWAP': [] ,'XMKT SWAP':[],'FX':[] ,'OTHER/CIX': []}
master = pd.DataFrame({})


In [5]:

x = ['-'] + [i for i in trades.keys()]
assetW = Dropdown(options = x,description = 'Instrument')

output1 = Output()
output2 = Output()
outputn = Output()

def build_data_list(asset):

    eugov = ['DBR','SPGB','FRTR','BTP','BTP-DBR','FRTR-DBR','SPGB-DBR','BTP-SPGB','FRTR-SPGB']

    
    style = {'description_width': 'initial'}
    b_or_i = {'-': Text(value = 'Choose Instrument',disabled = True),
        'SWAP':Dropdown(options = [i.upper() for i in spot_library.keys()], description = 'Currency'),
         'FUTURE':Dropdown(options = ['USD','EUR','GBP'], description = 'Currency'),
         'BOND':Dropdown(options = ['UST','DBR','SPGB','FRTR','BTP','UKT','BTP-DBR','FRTR-DBR','SPGB-DBR','BTP-SPGB','FRTR-SPGB'], description = 'Bond'),
         'BOND FUTURE': Dropdown(options = ['Yield', 'Price'], description = 'Field'),
        'MD SWAP': Dropdown(options = ['FOMC','ECB','MPC'], description = 'Central Bank'),
         'ASW':Dropdown(options = ['UST','DBR','SPGB','FRTR','BTP','UKT','BTP-DBR','FRTR-DBR','SPGB-DBR','BTP-SPGB','FRTR-SPGB'], description = 'Bond'),
              'FX': Text(description = 'Pair'),
              'XMKT SWAP':Dropdown(options = [i.upper() for i in spot_library.keys()], description = 'Currency'),
              'OTHER/CIX':Text(description = 'Ticker')}
    
    what_trade = {'-': Text(value = 'Choose Instrument',disabled = True),
                  'SWAP':Text(description = 'Structure'),
         'FUTURE':Text(description = 'Structure'),
         'BOND':Text(description = 'Structure'),
        'BOND FUTURE':Text(description = 'Structure'),
         'ASW':Text(description = 'Structure'),
        'MD SWAP': Text(description = 'Structure'),
        'XMKT SWAP':Text(description = 'Structure'),
                  'FX':Text(value = ' ',disabled = True),
        'OTHER/CIX':Dropdown(options = ['','Last Price','YTM'],description = 'Attribute')}
    
    
    what_trade_p = {'-': Text(value = 'Choose Instrument',disabled = True),
                  'SWAP':Dropdown(options = ['-']+swap_ops, description = 'Preset'),
         'FUTURE':Dropdown(options = ['-']+clr_dd, description = 'Preset'),
         'BOND':Dropdown(options = ['-']+bond_ops, description = 'Preset'),
        'BOND FUTURE':Text(value = '-',disabled = True),
         'ASW':Dropdown(options = ['-']+bond_ops, description = 'Preset'),
             'MD SWAP':Text(value = '-',disabled = True),
             'XMKT SWAP':Text(value = '-',disabled = True),
             'FX': Text(value = '-',disabled = True),
             'OTHER/CIX':Dropdown(options = ['-',"MR Presets"], description = 'Preset')}
    
    final = {'-': Text(value = 'Choose Instrument',disabled = True),
                  'SWAP':Text(value = '-',disabled = True),
         'FUTURE':Text(value = '-',disabled = True),
         'BOND':Text(value = '-',disabled = True),
        'BOND FUTURE':Text(value = '-',disabled = True),
         'ASW':Dropdown(options = ['ESTR','EURIBOR'] , description = 'Against (if EUGOV)',style=style),
             'MD SWAP':Text(value = '-',disabled = True),
             'XMKT SWAP':Dropdown(options = [i.upper() for i in spot_library.keys()], description = 'Against'),
             'FX': Text(value = '-',disabled = True),
             'OTHER/CIX':Text(description = 'Label')}
    
    include_in_list = Button(description = 'Add Structure')
    removefrom_list = Button(description = 'Remove Last')
    include__preset = Button(description = 'Add Preset')
    
    global trades
    
    
    def on_button_clicked1(b):
        with output1:
            if asset =='-' or what_trade[asset].value == '':
                pass
            else:
                if asset in ['XMKT SWAP','OTHER/CIX','ASW']:
                    trades[asset].append((b_or_i[asset].value,what_trade[asset].value,final[asset].value))
                elif asset == 'BOND FUTURE':
                    trades[asset].append((what_trade[asset].value ,b_or_i[asset].value))
                else:
                    trades[asset].append((b_or_i[asset].value,what_trade[asset].value))
                    
                assetW.value = '-'


    def on_button_clickedn(b):
        with outputn:
            if asset =='-' or what_trade_p[asset].value == '-':
                pass
            else:
                if asset == 'FUTURE':
                    for tup in fut_presets(b_or_i[asset].value,what_trade_p[asset].value):
                        trades[asset].append(tup)
                elif asset == 'SWAP':
                    for tup in swap_presets(b_or_i[asset].value,what_trade_p[asset].value):
                        trades[asset].append(tup)
                elif asset == 'BOND':
                    for tup in bond_spline_presets(b_or_i[asset].value,what_trade_p[asset].value):
                        trades[asset].append(tup)
                elif asset == 'ASW':
                    for tup in asw_spline_presets(b_or_i[asset].value,what_trade_p[asset].value,final[asset].value):
                        trades[asset].append(tup)
                        
                elif asset == 'OTHER/CIX':
                    custom_trades = custom_presets(what_trade_p[asset].value) 
                    for a in custom_trades.keys():
                        trades[a] += custom_trades[a]
                    
                assetW.value = '-'    
                                     
                
    def on_button_clicked2(b):
        with output2:
            if assetW == '-':
                pass
            else:
                trades[asset].pop()
                assetW.value = '-'
    
    
    
    display(HBox([b_or_i[asset],what_trade[asset],final[asset]]))
    display(what_trade_p[asset])
        
        
    include_in_list.on_click(on_button_clicked1)
    removefrom_list.on_click(on_button_clicked2)
    include__preset.on_click(on_button_clickedn)
    display(HBox([include_in_list,removefrom_list,include__preset]))
    
    max_len = max([len(trades[i]) for i in trades.keys()])
    y = {}
    for i in trades.keys():
        if i == 'XMKT SWAP':
            y[i] = [f'{i[0]}-{i[2]} {i[1]}' for i in trades[i]] + [float('nan')]*(max_len - len(trades[i]))
            
        elif i  =='ASW':# and i[0] in eugov:
            
            y[i] = [f'{i[0]}-{i[1]} v {i[2]}'if i[0] in eugov else f'{i[0]} {i[1]}' for i in trades[i] ] + [float('nan')]*(max_len - len(trades[i]))
        
        elif i == 'BOND FUTURE':
            y[i] = [j[0] for j in trades[i]] + [float('nan')]*(max_len - len(trades[i]))
            
            
        elif i == 'OTHER/CIX':
            y[i] = [f'{i[2]}' for i in trades[i]] + [float('nan')]*(max_len - len(trades[i]))
            
        else:
            y[i] = [f'{i[0]} {i[1]}' for i in trades[i]] + [float('nan')]*(max_len - len(trades[i]))
    
    display(pd.DataFrame(y).style.format('{}',na_rep = ''))
    
interactive(build_data_list,asset=assetW)


interactive(children=(Dropdown(description='Instrument', options=('-', 'SWAP', 'FUTURE', 'BOND', 'BOND FUTURE'…

In [6]:
output3 = Output()
get_data_frombb = Button(description = 'Pull BBG Data')
gen_from = DatePicker(value =dt(dt.today().year - 1, dt.today().month,dt.today().day),description = 'Start')
gen_to = DatePicker(value = dt.today(),description = 'End')
display(HBox([gen_from,gen_to]))
display(get_data_frombb,output3)

def on_button_clicked3(b):
    global master
    with output3:
        clear_output()
        master = function_for_bb(trades, gen_from.value,gen_to.value)
        dis = pd.DataFrame({},index = ['CURRENT', 'AVG MOVE', 'DAILY VOL'])
        dis.loc['CURRENT',master.columns] = master.loc[master.index[-1]]
        dis.loc['AVG MOVE',master.columns] = (master.diff().mean()) 
        dis.loc['DAILY VOL',master.columns] = (master.diff().std())

        display(dis.style.format('{:.2f}',na_rep = ''))
get_data_frombb.on_click(on_button_clicked3)    

Button(description='Pull BBG Data', style=ButtonStyle())

Output()

## CORRELATIONS 

In [7]:
output4 = Output()

rank = Button(description = 'Show Top Pairs')
run_corr = Button(description = 'Correlation Matrix')
Update3 = Button(description = 'Update Dropdown')

study_target = Dropdown(options = master.columns, description='Target')
num_pairs = IntText(value = 5,  description ='# Pairs')
val_or_diff = RadioButtons(options = ['Values', 'Differences'], value = 'Differences')
metric = ToggleButtons(options = ['Correlation', 'R-Squared', 'Beta'], description = 'Metric')
zoomer = IntSlider(min = 50, max = 200, step =5,value = 100, description = ' Zoom (%)')

focus = HBox([study_target,num_pairs])
buts = HBox([Update3,run_corr,rank])
display(VBox([focus,metric,val_or_diff,zoomer]))
display(buts,output4)


def on_button_clicked4(b):
    with output4:
        clear_output()
        new_master = master.copy(deep = True)
        new_cols = [study_target.value] + list(new_master.drop(columns = [study_target.value]).columns)  
        new_master = new_master[new_cols]
        if val_or_diff.value == 'Values':
            corr_matrix = new_master.corr()
        else:
            corr_matrix = new_master.diff().corr()
        mask = np.triu(np.ones_like(corr_matrix, dtype=np.bool))
        mask = mask[1:, :-1]
        corr = corr_matrix.iloc[1:,:-1].copy()
        fig, ax = plt.subplots(figsize=((zoomer.value * 20/ 100), (zoomer.value * 16/ 100)))
        sns.heatmap(corr,mask = mask,annot=True, fmt = '.1%',cmap='RdYlGn',linewidths=2, vmin=-1, vmax=1, cbar_kws={"shrink": .85})
        plt.yticks(rotation=0)
        ax.xaxis.tick_top() # x axis on top
        ax.xaxis.set_label_position('top')
        #title = 'CORRELATION MATRIX'
        #plt.title(title, loc='left', fontsize=(zoomer.value * 18/ 100))
        plt.show()

def on_button_clicked9(b):
    with output4:
        clear_output()
        if val_or_diff.value == 'Values':
            base1 = master
        else:
            base1 = master.diff()
        analyse = pd.DataFrame({})
        analyse['Correlation'] = base1.corr()[study_target.value]
        analyse['R-Squared'] = base1.corr()[study_target.value] ** 2
        analyse['Beta'] = base1.cov()[study_target.value]/base1.var()
        analyse.drop(axis=0,index  = study_target.value, inplace = True)
        analyse.sort_values(by = 'R-Squared',ascending = False,inplace = True)
        interim = analyse.head(num_pairs.value)
        disp = pd.DataFrame( interim[metric.value],columns =[metric.value],index = interim.index)
        fig, ax = plt.subplots(figsize=((zoomer.value * 20/ 100), (zoomer.value * 16/ 100)))
        ax = sns.heatmap(disp,annot=True, fmt = '.2f',
                         cmap='RdYlGn',linewidths=2, vmin=disp.min()[metric.value], vmax=disp.max()[metric.value],
                         cbar_kws={"shrink": .85},xticklabels = [study_target.value])
        plt.yticks(rotation=0)
        title = metric.value.upper()
        ax.xaxis.tick_top() # x axis on top
        ax.xaxis.set_label_position('top')
        plt.title(title, loc='left', fontsize=(zoomer.value * 18/ 100))
        plt.show()
        
        
def on_button_clicked10(b):
    study_target.options = ['-']+list(master.columns)
    
        
        
        
        
run_corr.on_click(on_button_clicked4)  
rank.on_click(on_button_clicked9)
Update3.on_click(on_button_clicked10)

Output()

## CHARTING

In [8]:
output5 = Output()
produce_chart = Button(description = 'Chart')
Update = Button(description = 'Update Dropdown(s)')
output6 = Output() 

which1 = Dropdown(options = master.columns, description='Series')
which2 = Dropdown(options = ['-']+list(master.columns), description='Against')
lag1 = IntText(value = 0, description = 'Lead/Lag 1st')
lag2 = IntText(value = 0, description = 'Lead/Lag 2nd')
to_plot_spread = Checkbox(description = 'Plot Spread')

display(VBox([HBox([which1,which2]),HBox([lag1,lag2]),to_plot_spread]))
display(Update)
display(produce_chart,output5)


def on_button_clicked5(b):
    with output5:
        clear_output()
        if which2.value == '-':
            fig, ax = plt.subplots(figsize = (18,9))
            ax.plot(master[which1.value].shift(lag1.value), color = 'b')
            ax.set_title(which1.value, fontsize =20)
            
        else:
            if to_plot_spread.value:
                fig,ax1 = plt.subplots(figsize = (20,9))
                spr = master[which1.value].shift(lag1.value) - master[which2.value].shift(lag2.value) 
                ax1.plot(spr,color = 'dodgerblue', label= 'Spread')
            else:
                fig,ax1 = plt.subplots(figsize = (20,9))
                ax2 = ax1.twinx()
                ax1.plot(master[which1.value].shift(lag1.value), color = 'b', label = which1.value)
                ax2.plot(master[which2.value].shift(lag2.value), color = 'r', label = which2.value)
                ax2.legend()

            if lag1.value == 0 and lag2.value == 0:
                dynamic_title = f"{which1.value} v. {which2.value}"
            elif lag1.value != 0 and lag2.value == 0:
                dynamic_title = f"{which1.value} ({'lagged' if lag1.value<0 else 'lead'} {abs(lag1.value)} days) v. {which2.value}"
            elif lag1.value == 0 and lag2.value != 0:
                dynamic_title = f"{which1.value}  v. {which2.value} ({'lagged' if lag2.value<0 else 'lead'} {abs(lag2.value)} days)"
            elif lag1.value != 0 and lag2.value != 0:
                dynamic_title = f"{which1.value} ({'lagged' if lag1.value<0 else 'lead'} {abs(lag1.value)} days) v. {which2.value} ({'lagged' if lag2.value<0 else 'lead'} {abs(lag2.value)} days)"
            ax1.set_title(dynamic_title, fontsize =20)

            
        im = image.imread('Y:\\Analytics and Tools\\Swaps\\Scrape Data\\logo_wm.png')
        fig.figimage(im, 425, 150, zorder=3, alpha=0.15)
        plt.show()

def on_button_clicked6(b):
    which1.options = master.columns
    which2.options = ['-']+list(master.columns)
    lag1.value = 0
    lag2.value = 0
    
produce_chart.on_click(on_button_clicked5)
Update.on_click(on_button_clicked6)

Button(description='Update Dropdown(s)', style=ButtonStyle())

Button(description='Chart', style=ButtonStyle())

Output()

##  ROLLING CORRELATION

In [9]:
output7 = Output()
output8 = Output() 

produce_rolling = Button(description = 'Rolling Correlation')
Update2 = Button(description = 'Update Dropdown')

which3 = Dropdown(options = master.columns, description='Series')
which4 = Dropdown(options = master.columns, description='Against')
window = IntText(value = 30, description = 'Window')

display(HBox([which3,which4,window]))
display(Update2)
display(produce_rolling,output7)


def on_button_clicked7(b):
    with output7:
        clear_output()
        rol = master[which3.value].rolling(window.value).corr(master[which4.value])
        fig, ax = plt.subplots(figsize = (18,9))
        #ax.hlines(rol.mean(),rol.index[0].strftime('%Y-%m-%d'),rol.index[-1].strftime('%Y-%m-%d'))                                                      
        ax.plot(rol, color = 'g')
        
        ax.set_title(f"{which3.value} v {which4.value} - {window.value} days Rolling Correlation", fontsize = 20)
        im = image.imread('Y:\\Analytics and Tools\\Swaps\\Scrape Data\\logo_wm.png')
        fig.figimage(im, 425, 150, zorder=3, alpha=0.15)
        plt.show()

def on_button_clicked8(b):
    which3.options = master.columns
    which4.options = master.columns
    window.value = 30

produce_rolling.on_click(on_button_clicked7)
Update2.on_click(on_button_clicked8)

Button(description='Update Dropdown', style=ButtonStyle())

Button(description='Rolling Correlation', style=ButtonStyle())

Output()

## REGRESSION

In [10]:
def adf(ser):
    if sm.tsa.adfuller(ser)[1] < 0.05:
        return 'No evidence of non-stationarity in model residuals '
    else:
        return 'Model residuals exhibit non-stationarity. Consider improving the specification with more/different\n explanatory factors'
    
gen_warning = 'This model is an backward-looking conditional expectations model and has not undergone robust train-test\n validation.'    
    
output11 = Output() 

regr = Button(description = 'Run Regression')
Update3 = Button(description = 'Update Inputs')

y_variable = Dropdown(options = master.columns, description='y')
X_variables = SelectMultiple(options = master.columns, description='X')
val_or_diff2 = RadioButtons(options = ['Values', 'Differences'])
add_a_constant = Checkbox(value = True, description = 'Add Constant')
show_what = ToggleButtons(options = ['Summary','Scatter','Model Plot','Residual Dist.','Diagnostics'], description = 'Result')

ui1 = VBox([y_variable,add_a_constant,val_or_diff2])
ui2 = HBox([ui1,X_variables])
ui3 = VBox([ui2,show_what])

def on_button_clicked11(b):
    with output11:
        clear_output()
        dset = master
        if val_or_diff2.value == 'Values':
            y = dset[y_variable.value]
            X = dset[list(X_variables.value)]
        else:
            y = dset[y_variable.value].diff().dropna()
            X = dset[list(X_variables.value)].diff().dropna()
        
        if add_a_constant.value:
            X = sm.add_constant(X)
        
        model = sm.OLS(y,X).fit(cov_type='HC3')
        fitted = model.predict(X)
        resid = model.resid
        
        stats = pd.DataFrame({'CURRENT': y.values[-1],'MODEL':fitted.values[-1],'RESID STD': resid.std(),'RICH/CHEAP':(y.values[-1]-fitted.values[-1])/(resid.std())},index =[y_variable.value])
        
        
        
        if show_what.value == 'Diagnostics':
            
            display(model.summary())
            print(adf(resid))
            print(gen_warning)
        elif show_what.value == 'Summary':
            display(stats.T.style.format('{:.2f}'))
        else:
            if show_what.value == 'Model Plot':

                    fig,(ax1,ax2) = plt.subplots(nrows = 2, ncols = 1)

                    plt.subplots_adjust(left=0, right=2)
                    plt.subplots_adjust(bottom=0, top=3)
                    
                    ax1.plot(fitted,color = 'red',label = 'Model')
                    ax1.plot(y,color = 'blue',label = 'Actual')
                    ax1.set_title(f"{y_variable.value} - MODEL PLOT",fontsize =20)
                    ax1.set_ylabel('bps')
                    ax1.legend()
                    
                    ax2.plot(resid,label='Residual',color = 'lightgreen')
                    ax2.set_title(f"{y_variable.value} - RESIDUAL PLOT",fontsize =20)
                    ax2.set_ylabel('bps')
                    im = image.imread('Y:\\Analytics and Tools\\Swaps\\Scrape Data\\logo_wm.png')
                    fig.figimage(im, 350, 575, zorder=3, alpha=0.15)
                    fig.figimage(im, 350, 100, zorder=3, alpha=0.15)
                    ax2.legend()
                    

                
            elif show_what.value == 'Scatter':
                if len(X_variables.value)>1:
                    print(f"Cannot display a 2-dimensional scatter for a {len(X_variables.value)+1}-dimensional regression")
                else:
                    fig,ax = plt.subplots(figsize=(18,9))
                    ax.scatter(dset[list(X_variables.value)].values[:-1],y.values[:-1])
                    ax.scatter(dset[list(X_variables.value)].values[-1],y.values[-1], color = 'darkblue',s =320, marker ='X')
                    ax.plot(dset[list(X_variables.value)].values,fitted.values, color ='r')
                    ax.set_ylabel(y_variable.value)
                    ax.set_xlabel(X_variables.value[0])
                    im = image.imread('Y:\\Analytics and Tools\\Swaps\\Scrape Data\\logo_wm.png')
                    fig.figimage(im, 425, 150, zorder=3, alpha=0.15)
                    ax.set_title(f"{y_variable.value} - SCATTER AND LINE OF BEST FIT", fontsize = 20)
                    

            elif show_what.value == 'Residual Dist.':
                fig,ax = plt.subplots(figsize=(18,9))
                ax.hist(resid, bins = 100)
                im = image.imread('Y:\\Analytics and Tools\\Swaps\\Scrape Data\\logo_wm.png')
                fig.figimage(im, 425, 150, zorder=3, alpha=0.15)
                ax.set_title("Distribution of Residuals".upper(),fontsize = 20)
                    
            plt.show()
        
        
def on_button_clicked12(b):
    y_variable.options = master.columns
    X_variables.options = master.columns
    X_variables.value = tuple([master.columns[1]])
    
    
display(ui3)
display(HBox([Update3,regr]),output11)


regr.on_click(on_button_clicked11)
Update3.on_click(on_button_clicked12)

Output()

## VaR - Value at Risk

In [11]:
output13 = Output()


over_period = {'1D':1,'1W':5,'2W':10,'1M':22,'3M':66}
produce_VaR = Button(description = 'Generate VaR')
zero_mean  = Checkbox(description = 'Assume zero mean')

Update4 = Button(description = 'Update Dropdown')


which5 = Dropdown(options = master.columns, description='Trade')
over = Dropdown(options = over_period.keys(), description='Over')
direction = ToggleButtons(options = ['PAY' , 'REC'], description = 'Direction')

display(VBox([HBox([which5,over]),direction,zero_mean]))
display(Update4)
display(produce_VaR,output13)


def on_button_clicked13(b):
    with output13:
        clear_output()

        true_avg = master[which5.value].diff().mean() * over_period[over.value]
        current = master[which5.value].values[-1]
        vol = master[which5.value].diff().std() * np.sqrt(over_period[over.value])
       
        if zero_mean.value:
            avg =0
            print('VaR is calculated assuming zero drift')
        else:
            avg = true_avg
        tile = [avg - vol*i if direction.value == 'PAY' else avg + vol*i  for i in (1.645,2.33)]
        sl = [current+i for i in tile]
        
        var_table = pd.DataFrame({'VaR Move': tile, 'Stop':sl}, index = [f'at {i}%' for i in (5,1)])
        display(var_table.style.format('{:.2f}'))
        display(pd.DataFrame({'SUMMARY':[current,true_avg,vol]}, index = ['Current','Average Move', 'Vol Move']).style.format('{:.2f}'))
         

def on_button_clicked14(b):
    with output13:
        clear_output()
        which5.options = master.columns
        over.value = '1D'

produce_VaR.on_click(on_button_clicked13)
Update4.on_click(on_button_clicked14)

Button(description='Update Dropdown', style=ButtonStyle())

Button(description='Generate VaR', style=ButtonStyle())

Output()